In [5]:
import sklearn
import pickle
import pandas as pd
import sklearn.linear_model
import optuna
import ML.features.features_with_sentence_list as features
from ML.natasha_func import ( get_doc, get_sents )
from tqdm import tqdm

In [75]:
dataset = pickle.load(open("dataset_1.pkl", "rb"))

In [80]:
authors = list(set(dataset['authors']))
dict_authors = dict()
reverse_dict_authors = dict()
for index in range(len(authors)):
    dict_authors[authors[index]] = index
    reverse_dict_authors[index] = authors[index]

target = []
for author in dataset["authors"]:
    target += [dict_authors[author]]

In [86]:
feature1 = []
feature2 = []
for book in tqdm(dataset['books']):
    doc = get_doc(book)
    sentences = get_sents(doc)

    feature1.append(features.avg_length_of_sentence_by_letters(sentences))
    feature2.append(features.capitalized_words_count_without_start_of_sentences(sentences))

100%|██████████| 100/100 [06:02<00:00,  3.62s/it]


In [151]:
!pip install bokeh

     ---------------------------------------- 8.3/8.3 MB 9.8 MB/s eta 0:00:00
     ---------------------------------------- 56.7/56.7 kB 2.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [152]:
from Text import Text

In [155]:
import json
import compress_pickle
import os


DATA_PATH = "C:\\Users\\User\\PycharmProjects\\texts_attribution\\Данные"
bridge = []
with open(os.path.join(DATA_PATH, "docs_info.json"), "r") as f_docs_info:
    docs_info = json.load(f_docs_info)
    authors = docs_info["authors"]
    file_names = docs_info["file_name"]

    for i, file_name in enumerate(tqdm(file_names)):
        author = authors[i]
        doc = compress_pickle.load(os.path.join(DATA_PATH, "Обработанные", file_name))
        sentences = get_sents(doc)
        text = Text(doc.text)
        bridge.append(text.empiric_bridge_test()[1])

100%|██████████| 516/516 [07:21<00:00,  1.17it/s]


In [156]:
bridge

[0.37125309982355775,
 2.0297375611496905e-05,
 0.33416062649111694,
 0.007238859377269612,
 4.3095909756919143e-13,
 0.022086010996770083,
 0.028884634599444844,
 1.5547494776384397e-09,
 0.30497610321543067,
 0.07533964117222096,
 0.1786045492007641,
 0.0010274720439936026,
 6.315312225919549e-08,
 0.003238400807080677,
 0.608971967923418,
 0.2008637100453935,
 0.01671383850386322,
 0.002227414296889676,
 2.2475978587981814e-05,
 0.00020506157327983724,
 0.00016620945724627687,
 0.0015297286623979715,
 0.0011325099915981327,
 1.1508452972377047e-05,
 0.16122901157288436,
 0.08205507616253117,
 0.0015590418627448022,
 0.11876849522821017,
 0.024651228009842867,
 0.013594671411377355,
 0.3759303717424433,
 0.9698452107505461,
 0.04532848033007894,
 0.15521918432637247,
 0.6946340398785876,
 0.021274534833979168,
 0.00525209366908865,
 0.0009915100730390816,
 0.8034133349599749,
 0.489561173127216,
 0.9264344297860742,
 0.028346197640195647,
 0.00161588094990248,
 0.009049862349453975,


In [107]:
df["bridge"] = bridge

NameError: name 'bridge' is not defined

In [107]:
df.rename(columns={"feature": "targets"}, inplace=True)

In [88]:
authors_seen = []
test_ixs = []
train_ixs = []
for ixs in range(len(df)):
    aut = df['targets'][ixs]
    if authors_seen.count(aut) < 2:
        authors_seen += [aut]
        test_ixs += [ixs]
    else: train_ixs += [ixs]

y_test = df.iloc[test_ixs]['targets']
x_test = df.iloc[test_ixs].drop(columns=["targets"])
y_train = df.iloc[train_ixs]['targets']
x_train = df.iloc[train_ixs].drop(columns=["targets"])

In [6]:
def validation(df, model):
    accuracy = []
    for shift in range(0, 10, 2):
        authors_seen = []
        test_ixs = []
        for ixs in range(len(df)):
            aut = df['targets'][ixs]
            if authors_seen.count(aut) < 2:
                authors_seen += [aut]
                test_ixs += [ixs + shift]
        train_ixs = list(set(range(len(df))) - set(test_ixs))
        y_test = df.iloc[test_ixs]['targets']
        x_test = df.iloc[test_ixs].drop(columns=["targets"])
        y_train = df.iloc[train_ixs]['targets']
        x_train = df.iloc[train_ixs].drop(columns=["targets"])

        rfc = model
        rfc.fit(x_train, y_train)
        accuracy.append(accuracy_score(y_test, rfc.predict(x_test)))
    return pd.Series(accuracy)

In [7]:
def cross_val(df, model):
    accuracy = []
    train_last, test_last = 0, 0
    for shift in range(0, 10, 2):
            authors_seen = []
            test_ixs = []
            for ixs in range(len(df)):
                aut = df['targets'][ixs]
                if authors_seen.count(aut) < 2:
                    authors_seen += [aut]
                    test_ixs += [ixs + shift]
            train_ixs = list(set(range(len(df))) - set(test_ixs))
            if shift != 8:
                y_test = df.iloc[test_ixs]['targets']
                x_test = df.iloc[test_ixs].drop(columns=["targets"])
                y_train = df.iloc[train_ixs]['targets']
                x_train = df.iloc[train_ixs].drop(columns=["targets"])

                rfc = model
                rfc.fit(x_train, y_train)
                accuracy.append(accuracy_score(y_test, rfc.predict(x_test)))
            else:
                train_last = train_ixs
                test_last = test_ixs

    return pd.Series(accuracy), train_last, test_last

In [8]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score


In [9]:
df = pd.read_csv("C:\\Users\\User\\PycharmProjects\\texts_attribution\\Данные\\first_dataset.csv")

In [10]:
df.rename(columns={func_name: func_name.split()[1] for func_name in df.columns[:-1]}, inplace=True)

In [11]:
df.rename(columns={"target": "targets"}, inplace=True)

In [176]:
def objective(trial):
    x = trial.suggest_int("n_estimators", 1, 500)
    y = trial.suggest_int("max_depth", 1, 100)
    z = trial.suggest_int("min_samples_split", 2, 100)
    w = trial.suggest_int("min_samples_leaf", 1, 100)
    etc = ExtraTreesClassifier(n_estimators=x, max_depth=y, min_samples_split=z, min_samples_leaf=w)
    return cross_val(df, etc)[0].mean()

In [177]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200)

[I 2023-07-22 23:57:18,720] A new study created in memory with name: no-name-816c5074-d0c6-42fb-b708-4bfc214399af
[I 2023-07-22 23:57:22,451] Trial 0 finished with value: 0.07 and parameters: {'n_estimators': 464, 'max_depth': 56, 'min_samples_split': 22, 'min_samples_leaf': 83}. Best is trial 0 with value: 0.07.
[I 2023-07-22 23:57:23,439] Trial 1 finished with value: 0.16749999999999998 and parameters: {'n_estimators': 109, 'max_depth': 5, 'min_samples_split': 5, 'min_samples_leaf': 53}. Best is trial 1 with value: 0.16749999999999998.
[I 2023-07-22 23:57:25,878] Trial 2 finished with value: 0.07250000000000001 and parameters: {'n_estimators': 400, 'max_depth': 53, 'min_samples_split': 73, 'min_samples_leaf': 85}. Best is trial 1 with value: 0.16749999999999998.
[I 2023-07-22 23:57:26,210] Trial 3 finished with value: 0.275 and parameters: {'n_estimators': 22, 'max_depth': 21, 'min_samples_split': 97, 'min_samples_leaf': 14}. Best is trial 3 with value: 0.275.
[I 2023-07-22 23:57:29,

In [178]:
best_params = study.best_params
found_x = best_params["n_estimators"]
found_y = best_params["max_depth"]
found_z = best_params["min_samples_split"]
found_w = best_params["min_samples_leaf"]
found_x, found_y, found_z, found_w

(450, 28, 2, 1)

In [12]:
etc = ExtraTreesClassifier(n_estimators=450, max_depth=28, min_samples_split=2, min_samples_leaf=1)

In [13]:
vals, train_ixs, test_ixs = cross_val(df, etc)

In [14]:
df_no_t = df.drop(columns="targets")


In [15]:
etc.fit(df_no_t.iloc[train_ixs], df["targets"].iloc[train_ixs])

ExtraTreesClassifier(max_depth=28, n_estimators=450)

In [20]:
accuracy_score(df.iloc[test_ixs]["targets"], etc.predict(df_no_t.iloc[test_ixs]))

0.64

In [71]:
from sklearn.ensemble import ExtraTreesClassifier

In [186]:
validation(df, ExtraTreesClassifier()).mean()

0.5840000000000001

In [151]:
df.drop(columns="frequency_of_longest_word", inplace=True)

In [ ]:
df.value_counts()

In [146]:
df.to_csv("first_dataset.csv", index=False)

In [125]:
import plotly.express as px

fig = px.imshow(df.drop(columns=["targets"]).corr(), text_auto=True)
fig.update_layout(height=1000)
fig.show()

In [133]:
df.columns.value_counts()

freq_word_from_adverbs                                2
avg_syllable_per_noun                                 1
freq_of_question_mark                                 1
freq_of_comma                                         1
freq_of_dot                                           1
avg_syllable_per_adverb                               1
avg_syllable_per_adjective                            1
avg_syllable_per_verb                                 1
capitalized_words_count_without_start_of_sentences    1
most_common_first_letter_in_sentences                 1
freq_word_from_verbs                                  1
freq_word_from_noun                                   1
average_lenght_of_verbs                               1
average_lenght_of_adverbs                             1
average_lenght_of_adjectives                          1
average_lenght_of_nouns                               1
freq_of_exclamation_mark                              1
freq_of_bracket                                 

In [ ]:
validation(df, )

In [79]:
/logreg.fit()

ValueError: Expected 2D array, got 1D array instead:
array=[ 67.49211356  83.09268293 135.44338967].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [74]:
logreg.predict()

array([1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0], dtype=int64)